# Sistema de recomendación de video juegos - Modelo K-NN 
# Sistema de Recomendación de Videojuegos


## Metodología

El sistema de recomendación se basa en el filtrado colaborativo, específicamente en el método de los vecinos más cercanos (K-NN). Este enfoque utiliza la matriz de utilidad, que representa las horas de juego acumuladas (playtime) por cada usuario para cada juego, como base para calcular las similitudes entre los juegos.

Para construir la matriz de utilidad, primero preprocesamos los datos de `users_items`, que contiene información sobre los juegos que los usuarios han jugado y las horas acumuladas en cada uno. Luego, transformamos estos datos en una matriz donde las filas representan los juegos (identificados por `item_id`) y las columnas representan a los usuarios. Los valores de la matriz son las horas jugadas, con ceros en caso de que un usuario no haya jugado un juego específico.

La matriz de utilidad se convierte en una matriz dispersa (sparse matrix) para mejorar la eficiencia computacional. Utilizamos el algoritmo K-NN para entrenar nuestro modelo de recomendación, configurándolo para utilizar la métrica del coseno, que es adecuada para medir similitudes en este tipo de datos.



In [1]:
import pandas as pd
import pickle
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

steam_games = pd.read_pickle("../data/steam_games.pkl")
users_items = pd.read_pickle("../data/users_items.pkl")

In [13]:
# reducir tamaño para modelo

users_items = users_items[:100000]


In [14]:
# Guardar para la API
steam_games.to_pickle("../data/steam_games_model.pkl")
users_items.to_pickle("../data/users_items_model.pkl")

In [4]:

# Crear una matriz de utilidad a partir de la tabla de usuarios e items
matrix_utility = users_items.pivot(
    index='item_id', columns='user_id', values='playtime_forever').fillna(0)

In [5]:
pickle.dump(matrix_utility, open("../data/matrix_utility.pkl", "wb"))

In [6]:
# Convertir la matriz de utilidad a una matriz sparse para mejorar la eficiencia
matrix_utility_sparse = csr_matrix(matrix_utility.values)

In [8]:
model_knn = NearestNeighbors(
    metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# Entrenar el modelo
model_knn.fit(matrix_utility_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [11]:
pickle.dump(model_knn, open("../data/model_knn.pkl", "wb"))

In [10]:
def recomendacion_juego(item_id, steam_games_df, matrix_utility, model_knn, k=5):
    item_id_str = str(item_id)

    if item_id_str not in steam_games_df['id'].astype(str).values:
        return f"El juego con ID {item_id} no fue encontrado."

    try:
        item_idx = matrix_utility.index.get_loc(item_id_str)
        distances, indices = model_knn.kneighbors(
            matrix_utility.iloc[item_idx, :].to_numpy().reshape(1, -1), n_neighbors=k+1)

        recommended_ids = [matrix_utility.index[i]
                           for i in indices.flatten()[1:]]
        recomendaciones = []

        for rec_id in recommended_ids:
            if rec_id in steam_games_df['id'].astype(str).values:
                nombre_juego = steam_games_df[steam_games_df['id'].astype(
                    str) == rec_id]['app_name'].iloc[0]
                recomendaciones.append({'id': rec_id, 'nombre': nombre_juego})
            else:
                recomendaciones.append(
                    {'id': rec_id, 'nombre': 'Nombre de juego no encontrado'})

        return recomendaciones
    except Exception as e:
        return str(e)  # Devolver el mensaje de error


recomendacion_juego("300", steam_games, matrix_utility, model_knn, k=5)

[{'id': '30', 'nombre': 'Day Of Defeat'},
 {'id': '276770', 'nombre': 'Nombre de juego no encontrado'},
 {'id': '266150', 'nombre': 'Lost Saga Na'},
 {'id': '294810', 'nombre': 'Blazblue Continuum Shift Extend'},
 {'id': '1200', 'nombre': 'Red Orchestra Ostfront 4145'}]